In [45]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [46]:
url = "https://myanimelist.net/topanime.php"
top = 5 #para limitar a lista

# LENDO A TABELA

In [47]:
anime_list = pd.read_html(url,header=0)[0] #Pega a tabela
anime_list = anime_list.drop(columns=["Your Score","Status"]) #Apaga as colunas que n vai precisar
anime_list = anime_list[:top] #limita a lista usando o valor da vaiavel top animes
anime_list

,Rank,Title,Score
0,1,Fullmetal Alchemist: Brotherhood TV (64 eps) ...,9.10
1,2,Steins;Gate TV (24 eps) Apr 2011 - Sep 2011 ...,9.07
2,3,Bleach: Sennen Kessen-hen TV (13 eps) Oct 20...,9.06
3,4,GintamaÂ° TV (51 eps) Apr 2015 - Mar 2016 6...,9.06
4,5,Shingeki no Kyojin Season 3 Part 2 TV (10 eps...,9.05


# ALTERANDO E CRIANDO AS COLUNAS A PARTIR DO TITLE (separando usando regex)

In [48]:
df_anime = pd.DataFrame(anime_list)
df_anime["Title"] =  [re.search(r"(\w.*)(TV|Movie|OVA|ONA|Special)",anime).group(1) for anime in anime_list["Title"]]
df_anime["Type"] =  [re.search(r"(\w.*)(TV|Movie|OVA|ONA|Special)",anime).group(2) for anime in anime_list["Title"]]
df_anime["Episodes"] =[re.search(r"(\d+)? eps",anime).group().strip("eps") for anime in anime_list["Title"]]
df_anime["Years"] = [re.search(r"(\w+ \d{4}) [-] (\w+ \d{4})?",anime).group() for anime in anime_list["Title"]]
df_anime["Members"] =  [re.search(r"(\d+,?\d+?,?\d+? members)",anime).group().strip("members") for anime in anime_list["Title"]]
df_anime

,Rank,Title,Score,Type,Episodes,Years,Members
0,1,Fullmetal Alchemist: Brotherhood,9.10,TV,64,Apr 2009 - Jul 2010,"3,205,585"
1,2,Steins;Gate,9.07,TV,24,Apr 2011 - Sep 2011,"2,463,346"
2,3,Bleach: Sennen Kessen-hen,9.06,TV,13,Oct 2022 - Dec 2022,"465,312"
3,4,GintamaÂ°,9.06,TV,51,Apr 2015 - Mar 2016,"602,306"
4,5,Shingeki no Kyojin Season 3 Part 2,9.05,TV,10,Apr 2019 - Jul 2019,"2,133,404"


# RECEBENDO E FILTRANDO OS LINKS DOS ANIMES

In [49]:
pagina = requests.get(url) # instanciando os dados da pagina

sopa = BeautifulSoup(pagina.content,"html.parser") # iniciando com o BS4

bloco_tabela = sopa.find("table") # procura a tabela

links = bloco_tabela.find_all("a",class_="hoverinfo_trigger fl-l ml12 mr8" ,href=True) # procurando a tag que esta o link na tabela
links = [link["href"] for link in links] # armazenando cada link numa lista

links=links[:top] #limitando a lista para o valor escolhido

# ENTRANDO EM CADA LINK PARA GUARDAR AS INFORMAÇÕES

In [50]:
matriz_etaria, generos_e_temas_bruto = [],[]

# Looping para ler cada link na lista receber os dados, adicionar na lista e ir para o proximo link
for url in links:
    pagina_anime = requests.get(url)
    body = BeautifulSoup(pagina_anime.content,"html.parser")
    bloco_side = body.find("div",class_="leftside")

    #corpo
    body = BeautifulSoup(pagina_anime.content,"html.parser")
    bloco_side = body.find("div",class_="leftside")


    try:
        #genero
        generos_e_temas = bloco_side.find_all("span",attrs={"itemprop": "genre"})
        generos_e_temas = [i.get_text() for i in generos_e_temas]

        #idade
        faixa_etaria = bloco_side.find(string=re.compile(r"\s[A-Za-z]{1,2}\s?-|[+]\s?")).text
        faixa_etaria = re.search(r"(\w.+?).+(-|[+])",faixa_etaria).group()
    except:
        generos = ""
        faixa_etaria = ""

    # matriz com os generos e temas misturados
    generos_e_temas_bruto.append(generos_e_temas)

    #Criadno a matriz faixa etária
    matriz_etaria.append(faixa_etaria)


# SEPARANDO OS GENEROS DOS TEMAS

In [62]:
matriz_genero,matriz_tema = [],[] #criando as tres lista vazias para inserir os dados da matriz

# Listas de temas e generos padrão disponibilzados no site
lista_de_generos =["Action","Adventure","Avant Garde","Award Winning","Boys Love","Comedy","Drama","Fantasy","Girls Love","Gourmet",
                   "Horror","Mystery","Romance","Sci-Fi","Slice of Life","Sports","Supernatural","Suspense"]

# separando os temas dos generos
for i in generos_e_temas_bruto:
  lista_gen,lista_tem =[],[]
  for genero in i:
    if genero in lista_de_generos:
      lista_gen.append(genero)
    else:
      lista_tem.append(genero)

  matriz_genero.append(", ".join(str(genero) for genero in lista_gen))
  matriz_tema.append(", ".join(str(tema) for tema in lista_tem))


# ADICIONANDO AO DF_ANIME

In [63]:
df_anime["Generos"] = matriz_genero  #criando a coluna generos e adicionando os dados
df_anime["Temas"] = matriz_tema #criando a coluna generos e adicionando os dados
df_anime["Faixa etaria"] = matriz_etaria #criando a coluna generos e adicionando os dados
df_anime

,Rank,Title,Score,Type,Episodes,Years,Members,Generos,Temas,Faixa etaria
0,1,Fullmetal Alchemist: Brotherhood,9.10,TV,64,Apr 2009 - Jul 2010,"3,205,585","Action, Adventure, Drama, Fantasy","Military, Shounen",R - 17+
1,2,Steins;Gate,9.07,TV,24,Apr 2011 - Sep 2011,"2,463,346","Drama, Sci-Fi, Suspense","Psychological, Time Travel",PG-13 -
2,3,Bleach: Sennen Kessen-hen,9.06,TV,13,Oct 2022 - Dec 2022,"465,312","Action, Adventure, Fantasy",Shounen,R - 17+
3,4,GintamaÂ°,9.06,TV,51,Apr 2015 - Mar 2016,"602,306","Action, Comedy, Sci-Fi","Gag Humor, Historical, Parody, Samurai, Shounen",PG-13 -
4,5,Shingeki no Kyojin Season 3 Part 2,9.05,TV,10,Apr 2019 - Jul 2019,"2,133,404","Action, Drama","Gore, Military, Survival, Shounen",R - 17+
